# Read mbd_dataset

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
#!pip install access_parser

In [ ]:
!dir v1

In [ ]:
import pandas as pd
import numpy as np
import torch

from torcheeg.io.eeg_signal import EEGSignalIO

## Path to dir with data (remember the last '/')
#path = "/eeg_data/BNCI2014-001/"
path = "C:/Users/ahmm9/Desktop/v1/"

## Establish connection to datafile
IO = EEGSignalIO(io_path=str(path), io_mode='lmdb')

## Read metadata dataframe
metadata = pd.read_csv(path + 'sample_metadata.tsv', sep='\t')

idxs = np.arange(len(metadata))

eeg = torch.FloatTensor(np.array([IO.read_eeg(str(i)) for i in idxs]))
print(eeg.shape)

In [ ]:
sample_meta = pd.read_csv(path+"sample_metadata.tsv", sep ="\t")

newdf = sample_meta.sort_values("arousal")
newdf

In [ ]:
emotion_mapping = {
    'sadness': 0, 'tenderness': 1, 'anger': 2, 'amusement': 3, 'joy': 4,
    'fear': 5, 'neutral': 6, 'disgust': 7, 'inspiration': 8
}

stim_valence_mapping = {
    'negative': 0, 'neutral': 1, 'positive': 2
}

# Apply mappings
newdf["emotion_encoded"] = newdf["emotion"].map(emotion_mapping)
newdf["stim_valence_encoded"] = newdf["stim_valence"].map(stim_valence_mapping)

# Print result
newdf


In [ ]:
newdf.shape[0]/3
class_1 = newdf[0:37254]
class_2 = newdf[37254:37254*2]
class_3 = newdf[37254*2:]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Split the dataset into three equal parts
split_size = newdf.shape[0] // 3
class_1 = newdf.iloc[:split_size]
class_2 = newdf.iloc[split_size:split_size*2]
class_3 = newdf.iloc[split_size*2:]

# Define equal bin edges based on the global min/max of arousal
arousal_min = newdf["arousal"].min()
arousal_max = newdf["arousal"].max()
bin_edges = np.linspace(arousal_min, arousal_max, num=30)  # Adjust num for bin count

# Plot histograms with the same bin edges
plt.figure(figsize=(10, 6))

sns.histplot(class_1["arousal"], bins=bin_edges, color='blue', alpha=0.5, label="Class 1")
sns.histplot(class_2["arousal"], bins=bin_edges, color='green', alpha=0.5, label="Class 2")
sns.histplot(class_3["arousal"], bins=bin_edges, color='red', alpha=0.5, label="Class 3")

# Formatting
plt.xlabel("Arousal Level")
plt.ylabel("Frequency")
plt.title("Histogram of Arousal Levels (Classes)")
plt.legend()
plt.show()


In [ ]:
newdf

In [ ]:
newdf.columns

In [ ]:
kept_newdf = newdf[["subject","joy", "tenderness", "inspiration", "amusement", "anger", "disgust", "fear", "sadness", 
                    "arousal", "valence", "familiarity", "liking", "emotion_encoded", "stim_valence_encoded"]]

kept_newdf

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only the desired columns (excluding encoded columns)
boxplot_vars = kept_newdf.drop(columns=["subject","emotion_encoded", "stim_valence_encoded"])

# Plot boxplots
plt.figure(figsize=(15, 8))
sns.boxplot(data=boxplot_vars)
plt.xticks(rotation=45)
plt.title("Boxplot distribution of Emotions")
plt.tight_layout()
plt.show()


In [ ]:
for i in kept_newdf.columns:
    max = np.max(kept_newdf[i].values)
    min = np.min(kept_newdf[i].values)
    print(f"{i} :", max, min)

In [ ]:
print("emotion_encoded", set(newdf["emotion_encoded"].values))
print("stim_valence_encoded", set(newdf["stim_valence_encoded"].values))

In [ ]:
kept_newdf

In [ ]:
import pandas as pd

# Split new_merged_df into 3 classes while preserving index
three_classes = kept_newdf.shape[0] // 3
class_1_m = kept_newdf.iloc[0:three_classes].copy()
class_2_m = kept_newdf.iloc[three_classes:three_classes*2].copy()
class_3_m = kept_newdf.iloc[three_classes*2:].copy()

# Create y_label lists
one_list = [1] * class_1_m.shape[0]
two_list = [2] * class_2_m.shape[0]
three_list = [3] * class_3_m.shape[0]

# Assign labels
class_1_m['y_label'] = one_list
class_2_m['y_label'] = two_list
class_3_m['y_label'] = three_list

# Concatenate while preserving original index
combined_df = pd.concat([class_1_m, class_2_m, class_3_m])

# Ensure the index matches the original dataset
assert all(combined_df.index == kept_newdf.index)

# Print check
print(combined_df.head())


In [ ]:
combined_df_sort =combined_df.sort_index()
combined_df_sort

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count occurrences of each arousal label per subject
subject_arousal_counts = combined_df_sort.groupby(['subject', 'y_label']).size().unstack()

# Plot the distribution
plt.figure(figsize=(15, 6))
subject_arousal_counts.plot(kind='bar', stacked=True, colormap='viridis', figsize=(15, 6))

# Labels and title
plt.xlabel("Subject")
plt.ylabel("Count of Arousal Labels")
plt.title("Distribution of Arousal Labels per Subject in Arousal Set")
plt.legend(title="Arousal Label")
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle="--", alpha=0.7)

plt.show()


In [ ]:
# Get unique subjects¨
from sklearn.model_selection import train_test_split

# Split randomly at the row level
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42, stratify=combined_df['y_label'])

# Check class distribution
print(train_df['y_label'].value_counts(normalize=True))  # Should be balanced
print(test_df['y_label'].value_counts(normalize=True))   # Should be balanced



In [ ]:
subject_set = set(train_df["subject"].values)
print(len(list(subject_set)))
print(subject_set)
subjects = set(list(combined_df["subject"]))
print(subjects)

In [ ]:
not_in_train = list(set(subjects).difference(subject_set))
print(not_in_train)

In [ ]:
from collections import defaultdict, Counter
unique_vals = defaultdict(list)
for i in list(subject_set):

    number_1 = train_df[train_df["subject"]==1]
    Counting = Counter(number_1["y_label"].values)
    unique_vals[i] = Counting
print(unique_vals)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count occurrences of each arousal label per subject
subject_arousal_counts = train_df.groupby(['subject', 'y_label']).size().unstack()

# Plot the distribution
plt.figure(figsize=(15, 6))
subject_arousal_counts.plot(kind='bar', stacked=True, colormap='viridis', figsize=(15, 6))

# Labels and title
plt.xlabel("Subject")
plt.ylabel("Count of Arousal Labels")
plt.title("Distribution of Arousal Labels per Subject in Train Set")
plt.legend(title="Arousal Label")
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle="--", alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count occurrences of each arousal label per subject
subject_arousal_counts_test = test_df.groupby(['subject', 'y_label']).size().unstack()

# Plot the distribution
plt.figure(figsize=(15, 6))
subject_arousal_counts_test.plot(kind='bar', stacked=True, colormap='viridis', figsize=(15, 6))

# Labels and title
plt.xlabel("Subject")
plt.ylabel("Count of Arousal Labels")
plt.title("Distribution of Arousal Labels per Subject in Test Set")
plt.legend(title="Arousal Label")
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle="--", alpha=0.7)

plt.show()


In [ ]:
train_indices_df = pd.DataFrame(list(train_df.index), columns=['index'])
test_indices_df = pd.DataFrame(list(test_df.index), columns=['index'])

# Save to CSV
#train_indices_df.to_csv("train_indices.csv", index=False)
#test_indices_df.to_csv("test_indices.csv", index=False)

In [ ]:
train_indeces_reread = pd.read_csv("train_indices.csv")
test_indeces_reread = pd.read_csv("test_indices.csv")

In [ ]:
test_indeces_reread

In [ ]:
len(test_indeces_reread.index)

In [ ]:
test_index_list = []
train_index_list = []
for i_train in train_indeces_reread.values:

    train_index_list.append(i_train[0])
#print(test_index_list)

for i_test in test_indeces_reread.values:
    #print(i[0])
    test_index_list.append(i_test[0])

new_train_df = combined_df.iloc[train_index_list]
new_test_df = combined_df.iloc[test_index_list]
print(len(train_index_list))
print(len(test_index_list))

In [ ]:
# Get the indices for each class
def eeg_tensor_and_y_label(dataframe):
    class_y_label_1 = dataframe[dataframe['y_label']==1] #= one_list
    class_y_label_2 = dataframe[dataframe['y_label']==2] #= one_list
    class_y_label_3 = dataframe[dataframe['y_label']==3] #= one_list
    #class_2_m['y_label'] = two_list
    #class_3_m['y_label'] = three_list



    idx_class_1 = class_y_label_1.index.values
    idx_class_2 = class_y_label_2.index.values
    idx_class_3 = class_y_label_3.index.values

    # Use these indices to extract the corresponding EEG tensors
    eeg_class_1 = eeg[idx_class_1]
    eeg_class_2 = eeg[idx_class_2]
    eeg_class_3 = eeg[idx_class_3]
    # Step 1: Create label arrays
    labels_class_1 = [0] * eeg_class_1.shape[0]
    labels_class_2 = [1] * eeg_class_2.shape[0]
    labels_class_3 = [2] * eeg_class_3.shape[0]



    # Assuming we have corresponding index lists for each dataset

    # Step 2: Combine tensors and labels into list of tuples (tensor, label)
    dataset_class_1 = [(eeg_class_1[i], labels_class_1[i]) for i in range(eeg_class_1.shape[0])]
    dataset_class_2 = [(eeg_class_2[i], labels_class_2[i]) for i in range(eeg_class_2.shape[0])]
    dataset_class_3 = [(eeg_class_3[i], labels_class_3[i]) for i in range(eeg_class_3.shape[0])]

    dataset_mapping = {} 
    # Convert lists into a dictionary using their indexes
    dataset_mapping.update(dict(zip(idx_class_1, dataset_class_1)))
    dataset_mapping.update(dict(zip(idx_class_2, dataset_class_2)))
    dataset_mapping.update(dict(zip(idx_class_3, dataset_class_3)))

    # Retrieve samples in the exact order of dataframe.index
    full_dataset = [dataset_mapping[idx] for idx in dataframe.index if idx in dataset_mapping]

    return full_dataset

    # Step 3: Check
    print(len(dataset_class_1))
    print(len(dataset_class_2))
    print(len(dataset_class_3))
    print(len(full_dataset))
    return full_dataset


train_tensor = eeg_tensor_and_y_label(new_train_df)
test_tensor = eeg_tensor_and_y_label(new_test_df)

torch.save(train_tensor, "emotion_train_set.pt")
torch.save(test_tensor, "emotion_test_set.pt")



In [ ]:
for i in range(len(test_tensor)):
    print(test_tensor[i][1])

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
# Define a method for training one epoch
from torch.nn import Module
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader



def train_one_epoch(
        dataloader: DataLoader, model: Module, loss_fn, optimizer,
        scheduler: LRScheduler, epoch: int, device, print_batch_stats=True
):
    model.train()  # Set the model to training mode
    train_loss, correct = 0, 0

    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader),
                        disable=not print_batch_stats)

    for batch_idx, (X, y) in progress_bar:
        X, y = X.to(device), y.to(device)
        #print(X.shape)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()  # update the model weights
        optimizer.zero_grad()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).sum().item()

        #if print_batch_stats:
        #    progress_bar.set_description(
        #        f"Epoch {epoch}/{n_epochs}, "
        #        f"Batch {batch_idx + 1}/{len(dataloader)}, "
        #        f"Loss: {loss.item():.6f}"
        #    )

    # Update the learning rate
    scheduler.step()

    correct /= len(dataloader.dataset)
    return train_loss / len(dataloader), correct


In [ ]:
from collections import defaultdict
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np

@torch.no_grad()
def test_model(dataloader: DataLoader, model: torch.nn.Module, loss_fn, print_batch_stats=True):
    device = next(model.parameters()).device  # Get model device
    size = len(dataloader.dataset)
    n_batches = len(dataloader)
    model.eval()  # Switch to evaluation mode
    test_loss, correct = 0, 0

    # Initialize dictionaries for per-class tracking
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    # Lists to store true and predicted labels for confusion matrix
    all_preds = []
    all_targets = []

    if print_batch_stats:
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    else:
        progress_bar = enumerate(dataloader)

    for batch_idx, (X, y) in progress_bar:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        batch_loss = loss_fn(pred, y).item()

        test_loss += batch_loss
        correct += (pred.argmax(1) == y).sum().item()

        # Store predictions and true labels for confusion matrix
        all_preds.append(pred.argmax(1).cpu())
        all_targets.append(y.cpu())

        # Compute per-class accuracy
        preds_labels = pred.argmax(1)
        for label, pred_label in zip(y, preds_labels):
            class_total[label.item()] += 1
            class_correct[label.item()] += (label == pred_label).item()

        if print_batch_stats:
            progress_bar.set_description(
                f"Batch {batch_idx + 1}/{len(dataloader)}, Loss: {batch_loss:.6f}"
            )

    # Convert lists to tensors
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)

    # Compute per-class accuracy
    class_accuracies = {
        cls: (class_correct[cls] / class_total[cls]) * 100 if class_total[cls] > 0 else 0
        for cls in class_total
    }

    # Compute overall accuracy
    test_loss /= n_batches
    overall_accuracy = (correct / size) * 100

    # Print per-class accuracy
    #print("\nClass-wise Accuracy:")
    #for cls, acc in class_accuracies.items():
    #    print(f"  Class {cls}: {acc:.2f}%")

    #print(f"Test Accuracy: {overall_accuracy:.1f}%, Test Loss: {test_loss:.6f}\n")

    return test_loss, overall_accuracy, class_accuracies, all_preds, all_targets


In [ ]:
#!dir ..\

In [ ]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True

In [ ]:
#eeg.shape[1]

In [ ]:
from collapsed_shallow_fbscp import ShallowFBCSPNet


import collapsed_shallow_fbscp
import importlib
importlib.reload(collapsed_shallow_fbscp)
# Now you can access the updated class
from collapsed_shallow_fbscp import ShallowFBCSPNet

model = ShallowFBCSPNet(
    n_chans=eeg.shape[1],
    n_subjs=9,
    n_outputs=3,
    n_times=eeg.shape[2],
    #final_conv_length="auto",
    #edge_index = adjacency_matrix
)

# Display torchinfo table describing the model
print(model)

# Send model to GPU
if cuda:
    model.cuda()

# 

In [ ]:
import torch
import wandb
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import CrossEntropyLoss
import numpy as np

# Initialize Weights & Biases
wandb.init(project="Master Thesis", name="ShallowFBCSPNet Accuracy")

# Define hyperparameters
lr = 0.0001
weight_decay = 1e-4
batch_size = 124  # Start with 124
n_epochs = 20

# Log hyperparameters to wandb
wandb.config.update({
    "learning_rate": lr,
    "weight_decay": weight_decay,
    "batch_size": batch_size,
    "epochs": n_epochs
})

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs - 1)

# Define loss function
loss_fn = CrossEntropyLoss()

# Create DataLoaders
train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_tensor, batch_size=batch_size)

# Initialize lists to store all predictions & targets
all_preds, all_targets = [], []

# Training loop
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}: ", end="")

    train_loss, train_accuracy = train_one_epoch(
        train_loader, model, loss_fn, optimizer, scheduler, epoch, device
    )

    test_loss, test_accuracy, class_accuracies, batch_preds, batch_targets = test_model(test_loader, model, loss_fn)

    # Store predictions & labels for confusion matrix
    all_preds.extend(batch_preds)
    all_targets.extend(batch_targets)

    # Print class-wise accuracy
    print("\nClass-wise Accuracy:")
    for class_idx, acc in class_accuracies.items():
        print(f"  Class {class_idx}: {acc:.2f}%")

    # Log results to wandb
    wandb.log({
        "epoch": epoch,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy * 100,
        "test_loss": test_loss,
        "test_accuracy": test_accuracy,
        "learning_rate": scheduler.get_last_lr()[0],
        **{f"class_{class_idx}_accuracy": acc for class_idx, acc in class_accuracies.items()}
    })

    print(
        f"Train Accuracy: {100 * train_accuracy:.2f}%, "
        f"Average Train Loss: {train_loss:.6f}, "
        f"Test Accuracy: {test_accuracy:.2f}%, "
        f"Average Test Loss: {test_loss:.6f}\n"
    )

# Convert lists to NumPy arrays
all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

# Save predictions & true labels for later use (confusion matrix)
wandb.log({"all_preds": all_preds.tolist(), "all_targets": all_targets.tolist()})

wandb.finish()


In [ ]:
import numpy as np
import pandas as pd
import itertools
import scipy.stats as stats  # Corrected import
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Step 1: Simulate a 4-class dataset
np.random.seed(123)
classes = ["A", "B", "C", "D"]
observed = np.random.choice(classes, 100)  # True labels
predicted = np.random.choice(classes, 100)  # Predicted labels

# Create confusion matrix
cm = confusion_matrix(observed, predicted, labels=classes)
cm_df = pd.DataFrame(cm, index=classes, columns=classes)

print("Confusion Matrix:")
print(cm_df)

# Step 2: Classification Report
print("\nClassification Report:")
print(classification_report(observed, predicted, target_names=classes))

# Step 3: Apply One-vs-All (OvA) approach
def epi_tests_ova(cm, target_class, class_labels):
    idx = class_labels.index(target_class)
    
    TP = cm[idx, idx]
    FP = sum(cm[idx, :]) - TP
    FN = sum(cm[:, idx]) - TP
    TN = cm.sum() - (TP + FP + FN)
    
    contingency_table = np.array([[TP, FP], [FN, TN]])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)  # Fixed function call
    
    return {"Class": target_class, "TP": TP, "FP": FP, "FN": FN, "TN": TN, "Chi2": chi2, "p-value": p}

# Apply epi_tests_ova for each class
results = [epi_tests_ova(cm, cls, classes) for cls in classes]

# Step 4: Convert Results into a Summary Table
summary_table = pd.DataFrame(results)

# Round numeric values
summary_table.iloc[:, 1:] = summary_table.iloc[:, 1:].apply(lambda x: np.round(x, 2))

print("\nSummary Table of Metrics:")
print(summary_table)

# Step 5: Export Results to CSV
summary_table.to_csv("multi_class_metrics.csv", index=False)

# Step 6: Plot Confusion Matrix
plt.figure(figsize=(6, 5))
plt.imshow(cm, cmap="Blues", interpolation="nearest")
plt.colorbar()
plt.xticks(ticks=np.arange(len(classes)), labels=classes)
plt.yticks(ticks=np.arange(len(classes)), labels=classes)

# Add text annotations
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center", color="black")

plt.xlabel("Predicted")
plt.ylabel("Observed")
plt.title("Confusion Matrix")
plt.show()
